# Word Count

### Counting the number of occurances of words in a text is a popular first exercise using map-reduce.

## The Task
**Input:** A text file consisisting of words separated by spaces.  
**Output:** A list of words and their counts, sorted from the most to the least common.

We will use the book "Moby Dick" as our input.

In [0]:
#start the SparkContext
from pyspark import SparkContext


In [0]:
sc=SparkContext(master="local[4]")

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<command-3704061629036308> in <module>
----> 1 sc=SparkContext(master="local[4]")

/databricks/spark/python/pyspark/context.py in __init__(self, master, appName, sparkHome, pyFiles, environment, batchSize, serializer, conf, gateway, jsc, profiler_cls)
    143                 " is not allowed as it is a security risk.")
    144 
--> 145         SparkContext._ensure_initialized(self, gateway=gateway, conf=conf)
    146         try:
    147             self._do_init(master, appName, sparkHome, pyFiles, environment, batchSize, serializer,

/databricks/spark/python/pyspark/context.py in _ensure_initialized(cls, instance, gateway, conf)
    356 
    357                     # Raise error if there is already a running Spark context
--> 358                     raise ValueError(
    359                         "Cannot run multiple SparkContexts a

### Setup a plan for pretty print

In [0]:
def pretty_print_plan(rdd):
    for x in rdd.toDebugString().decode().split('\n'):
        print(x)

### Use `textFile()` to read the text

In [0]:
%%time
text_file = sc.textFile("/FileStore/tables/Moby_Dick.txt")

CPU times: user 773 µs, sys: 203 µs, total: 976 µs
Wall time: 74.8 ms


In [0]:
type(text_file)

Out[11]: pyspark.rdd.RDD

## Steps for counting the words

* split line by spaces.
* map `word` to `(word,1)`
* count the number of occurances of each word.

In [0]:
%%time
words =text_file.flatMap(lambda line: line.split(" "))
not_empty = words.filter(lambda x: x!='') 
key_values= not_empty.map(lambda word: (word, 1)) 
key_values.collect()
counts=key_values.reduceByKey(lambda a, b: a + b)

CPU times: user 90.7 ms, sys: 16.7 ms, total: 107 ms
Wall time: 1.65 s


In [0]:
counts.take(5)

Out[13]: [('The', 549), ('Project', 79), ('EBook', 1), ('of', 6587), ('Moby', 79)]

### flatMap()
Note the line:
```python
words =     text_file.flatMap(lambda line: line.split(" "))
```
Why are we using `flatMap`, rather than `map`?

The reason is that the operation `line.split(" ")` generates a **list** of strings, so had we used `map` the result would be an RDD of lists of words. Not an RDD of words.

The difference between `map` and `flatMap` is that the second expects to get a list as the result from the map and it **concatenates** the lists to form the RDD.

## The execution plan
In the last cell we defined the execution plan, but we have not started to execute it.

* Preparing the plan took ~100ms, which is a non-trivial amount of time, 
* But much less than the time it will take to execute it.
* Lets have a look a the execution plan.

### Understanding the details
To see which step in the plan corresponds to which RDD we print out the execution plan for each of the RDDs.  

Note that the execution plan for `words`, `not_empty` and `key_values` are all the same.

In [0]:
pretty_print_plan(text_file)

(2) /FileStore/tables/Moby_Dick.txt MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /FileStore/tables/Moby_Dick.txt HadoopRDD[8] at textFile at NativeMethodAccessorImpl.java:0 []


In [0]:
pretty_print_plan(words)

(2) PythonRDD[16] at RDD at PythonRDD.scala:58 []
 |  /FileStore/tables/Moby_Dick.txt MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /FileStore/tables/Moby_Dick.txt HadoopRDD[8] at textFile at NativeMethodAccessorImpl.java:0 []


In [0]:
pretty_print_plan(not_empty)

(2) PythonRDD[17] at RDD at PythonRDD.scala:58 []
 |  /FileStore/tables/Moby_Dick.txt MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /FileStore/tables/Moby_Dick.txt HadoopRDD[8] at textFile at NativeMethodAccessorImpl.java:0 []


In [0]:
pretty_print_plan(key_values)

(2) PythonRDD[10] at collect at <timed exec>:4 []
 |  /FileStore/tables/Moby_Dick.txt MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /FileStore/tables/Moby_Dick.txt HadoopRDD[8] at textFile at NativeMethodAccessorImpl.java:0 []


In [0]:
pretty_print_plan(counts)

(2) PythonRDD[18] at RDD at PythonRDD.scala:58 []
 |  MapPartitionsRDD[14] at mapPartitions at PythonRDD.scala:183 []
 |  ShuffledRDD[13] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[12] at reduceByKey at <timed exec>:5 []
    |  PythonRDD[11] at reduceByKey at <timed exec>:5 []
    |  /FileStore/tables/Moby_Dick.txt MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:0 []
    |  /FileStore/tables/Moby_Dick.txt HadoopRDD[8] at textFile at NativeMethodAccessorImpl.java:0 []


| Execution plan   | RDD |  Comments |
| :---------------------------------------------------------------- | :------------: | :--- |
|`(2)_PythonRDD[6] at RDD at PythonRDD.scala:48 []`| **counts** | Final RDD|
|`_/__MapPartitionsRDD[5] at mapPartitions at PythonRDD.scala:436 []`| **---"---** |
|`_/__ShuffledRDD[4] at partitionBy at NativeMethodAccessorImpl.java:0 [`| **---"---** | RDD is partitioned by key |
|`_+-(2)_PairwiseRDD[3] at reduceByKey at <timed exec>:4 []`| **---"---** | Perform mapByKey |
|`____/__PythonRDD[2] at reduceByKey at <timed exec>:4 []`| **words, not_empty, key_values** | The result of  partitioning into words|
| | |  removing empties, and making into (word,1) pairs|
|`____/__../../Data/Moby-Dick.txt MapPartitionsRDD[1] at textFile at Nat`| **text_file** | The partitioned text |
|`____/__../../Data/Moby-Dick.txt HadoopRDD[0] at textFile at NativeMeth`| **---"---** | The text source |

## Execution
Finally we count the number of times each word has occured.
Now, finally, the Lazy execution model finally performs some actual work, which takes a significant amount of time.

In [0]:
%%time
## Run #1
Count=counts.count()  # Count = the number of different words
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y) # 
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 15.4 ms, sys: 1.97 ms, total: 17.3 ms
Wall time: 566 ms


### Amortization
When the same commands are performed repeatedly on the same data, the execution time tends to decrease in later executions.

The cells below are identical to the one above, with one exception at `Run #3`

Observe that `Run #2` take much less time that `Run #1`. Even though no `cache()` was explicitly requested. The reason is that Spark caches (or materializes) `key_values`, before executing `reduceByKey()` because performng reduceByKey requires a shuffle, and a shuffle requires that the input RDD is materialized. In other words, sometime caching happens even if the programmer did not ask for it.

In [0]:
%%time
## Run #2
Count=counts.count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 13.3 ms, sys: 395 µs, total: 13.7 ms
Wall time: 422 ms


### Explicit Caching
In `Run #3` we explicitly ask for `counts` to be cached. This will reduce the execution time in the following run `Run #4` by a little bit, but not by much.

In [0]:
%%time
## Run #3, cache
Count=counts.cache().count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 11.9 ms, sys: 3.08 ms, total: 15 ms
Wall time: 625 ms


In [0]:
%%time
#Run #4
Count=counts.count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 7.79 ms, sys: 4.87 ms, total: 12.7 ms
Wall time: 295 ms


In [0]:
%%time
#Run #5
Count=counts.count()
Sum=counts.map(lambda x:x[1]).reduce(lambda x,y:x+y)
print('Different words=%5.0f, total words=%6.0f, mean no. occurances per word=%4.2f'%(Count,Sum,float(Sum)/Count))

Different words=33781, total words=215133, mean no. occurances per word=6.37
CPU times: user 8.87 ms, sys: 3.7 ms, total: 12.6 ms
Wall time: 315 ms
